# V24: TRICERATOPS FPP sensitivity to preprocessing/depth

Runs a small grid of **reasonable** preprocessing choices (detrend parameters) and records:
- the depth implied by each choice, and
- TRICERATOPS(+ ) FPP/NFPP (with and without AO/contrast-curve constraints).

This is not intended to optimize FPP; it is intended to evaluate whether the conclusion is stable under reasonable analysis choices.


In [ ]:
from __future__ import annotations

import contextlib
import json
import sys
from dataclasses import asdict
from pathlib import Path

import numpy as np

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

btv = sh.btv

step_id = '38_v24_triceratops_fpp_sensitivity_grid'
run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)

sectors = [82, 83]

# AO / contrast curve
contrast_curve_path = Path('docs/tutorials/data/tic188646744/PHARO_Kcont_plot.tbl')
contrast_curve = btv.load_contrast_curve_exofop_tbl(contrast_curve_path, filter='Kcont')

# Small grid of reasonable detrend configurations.
grid = [
    {'bin_hours': 4.0, 'duration_buffer_factor': 2.0, 'sigma_clip': 5.0},
    {'bin_hours': 6.0, 'duration_buffer_factor': 2.0, 'sigma_clip': 5.0},
    {'bin_hours': 12.0, 'duration_buffer_factor': 2.0, 'sigma_clip': 5.0},
    {'bin_hours': 6.0, 'duration_buffer_factor': 1.5, 'sigma_clip': 5.0},
]

# TRICERATOPS preset overrides for runtime stability in a grid.
overrides = {
    'mc_draws': 10000,
    'max_points': 1500,
    'window_duration_mult': 2.0,
}

base = sh.load_dataset()

# TRICERATOPS init can hit upstream TRILEGAL form changes. To keep this tutorial
# stable/reproducible, we reuse the cached TRICERATOPS Target object produced in V22.
seed_target_cache_src = Path(
    'persistent_cache/tutorial_toi-5807-incremental/36_v22_triceratops_fpp/'
    'cache_pdcsap_detrended/triceratops/target_cache/tic_188646744__sectors_82-83.pkl'
)
seed_trilegal_csv_src = Path(
    'persistent_cache/tutorial_toi-5807-incremental/36_v22_triceratops_fpp/'
    'cache_pdcsap_detrended/triceratops/188646744_TRILEGAL.csv'
)

def seed_target_cache(cache_dir: Path) -> None:
    tc_dir = cache_dir / 'triceratops' / 'target_cache'
    tc_dir.mkdir(parents=True, exist_ok=True)
    if seed_target_cache_src.exists():
        (tc_dir / seed_target_cache_src.name).write_bytes(seed_target_cache_src.read_bytes())
    tr_dir = cache_dir / 'triceratops'
    tr_dir.mkdir(parents=True, exist_ok=True)
    if seed_trilegal_csv_src.exists():
        (tr_dir / seed_trilegal_csv_src.name).write_bytes(seed_trilegal_csv_src.read_bytes())

def _run_one(*, detrend: dict, use_contrast_curve: bool) -> dict:
    # Step-local cache so we can cache detrended LCs under flux_type='pdcsap'.
    sc = detrend['sigma_clip']
    sc_s = 'none' if sc is None else str(sc)
    tag = (
        f"bh{detrend['bin_hours']:g}_bf{detrend['duration_buffer_factor']:g}_sc{sc_s}"
    ).replace('.', 'p')
    cache_dir = run_out_dir / f"cache_{tag}_{'cc' if use_contrast_curve else 'no_cc'}"
    seed_target_cache(cache_dir)

    detrended_by_sector: dict[int, btv.LightCurve] = {}
    for sector in sectors:
        detrended_by_sector[int(sector)] = sh.detrend_transit_masked_bin_median(
            base.lc_by_sector[int(sector)],
            bin_hours=float(detrend['bin_hours']),
            duration_buffer_factor=float(detrend['duration_buffer_factor']),
            sigma_clip=detrend['sigma_clip'],
        )

    from tess_vetter.api.datasets import LocalDataset

    ds_det = LocalDataset(
        schema_version=base.schema_version,
        root=base.root,
        lc_by_sector=detrended_by_sector,
        tpf_by_sector={},
        artifacts=base.artifacts,
    )

    cache = btv.hydrate_cache_from_dataset(
        dataset=ds_det,
        tic_id=sh.TIC_ID,
        flux_type='pdcsap',
        cache_dir=cache_dir,
        sectors=sectors,
    )

    stitched_det = sh.stitch_pdcsap_sectors(ds_det, sectors)
    depth_ppm, depth_err_ppm = sh.estimate_depth_ppm(stitched_det)

    stdout_path = run_out_dir / f"triceratops_{tag}_{'cc' if use_contrast_curve else 'no_cc'}_stdout.log"
    stderr_path = run_out_dir / f"triceratops_{tag}_{'cc' if use_contrast_curve else 'no_cc'}_stderr.log"

    with (
        stdout_path.open('w', encoding='utf-8') as out,
        stderr_path.open('w', encoding='utf-8') as err,
        contextlib.redirect_stdout(out),
        contextlib.redirect_stderr(err),
    ):
        result = btv.calculate_fpp(
            cache=cache,
            tic_id=sh.TIC_ID,
            period=sh.PERIOD_DAYS,
            t0=sh.T0_BTJD,
            depth_ppm=float(depth_ppm),
            duration_hours=sh.DURATION_HOURS,
            sectors=sectors,
            stellar_radius=sh.STELLAR.radius,
            stellar_mass=sh.STELLAR.mass,
            preset='fast',
            overrides=overrides,
            replicates=1,
            seed=42,
            contrast_curve=(contrast_curve if use_contrast_curve else None),
        )

    return {
        'detrend': detrend,
        'use_contrast_curve': bool(use_contrast_curve),
        'depth_ppm_used': float(depth_ppm),
        'depth_err_ppm_est': float(depth_err_ppm),
        'fpp_result': result,
        'logs': {'stdout': str(stdout_path), 'stderr': str(stderr_path)},
    }


rows: list[dict] = []
for detrend in grid:
    rows.append(_run_one(detrend=detrend, use_contrast_curve=False))
    rows.append(_run_one(detrend=detrend, use_contrast_curve=True))

payload = {
    'sectors': sectors,
    'grid': grid,
    'overrides': overrides,
    'contrast_curve': {
        'path': str(contrast_curve_path),
        'filter': str(getattr(contrast_curve, 'filter', None)),
        'n_points': int(len(contrast_curve.separation_arcsec)),
    },
    'rows': rows,
}

json_name = 'triceratops_fpp_sensitivity_grid.json'
(run_out_dir / json_name).write_text(json.dumps(payload, indent=2, sort_keys=True))
if docs_out_dir is not None:
    (docs_out_dir / json_name).write_text(json.dumps(payload, indent=2, sort_keys=True))

def _summ(r: dict) -> dict:
    out = {
        'use_contrast_curve': bool(r['use_contrast_curve']),
        'bin_hours': r['detrend']['bin_hours'],
        'duration_buffer_factor': r['detrend']['duration_buffer_factor'],
        'sigma_clip': r['detrend']['sigma_clip'],
        'depth_ppm_used': round(float(r['depth_ppm_used']), 3),
        'fpp': r['fpp_result'].get('fpp'),
        'nfpp': r['fpp_result'].get('nfpp'),
        'disposition': r['fpp_result'].get('disposition'),
    }
    return out

summary = [_summ(r) for r in rows]
print(json.dumps({'n_runs': len(rows), 'rows': summary}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

This notebook runs multiple TRICERATOPS instances; numerical results can vary slightly. The key expectations are:
- each run succeeds (no `cache_miss` / no `timeout`), and
- you see how FPP changes with depth/preprocessing and with AO constraints.

The full output is written to:
- `docs/tutorials/artifacts/tutorial_toi-5807-incremental/38_v24_triceratops_fpp_sensitivity_grid/triceratops_fpp_sensitivity_grid.json`

</details>


In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

step_id = '38_v24_triceratops_fpp_sensitivity_grid'
fname = 'V24_triceratops_fpp_sensitivity.png'

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
run_path = run_out_dir / fname
docs_path = (docs_out_dir / fname) if docs_out_dir is not None else None

def _label(d: dict) -> str:
    sc = d['sigma_clip']
    sc_s = 'none' if sc is None else str(sc)
    return f"bh={d['bin_hours']:g}h bf={d['duration_buffer_factor']:g} sc={sc_s}"

cfgs = grid

depth_no = []
depth_cc = []
fpp_no = []
fpp_cc = []

for d in cfgs:
    r0 = next(r for r in rows if (r['detrend'] == d and r['use_contrast_curve'] is False))
    r1 = next(r for r in rows if (r['detrend'] == d and r['use_contrast_curve'] is True))
    depth_no.append(float(r0['depth_ppm_used']))
    depth_cc.append(float(r1['depth_ppm_used']))
    fpp_no.append(float(r0['fpp_result'].get('fpp')) if r0['fpp_result'].get('fpp') is not None else np.nan)
    fpp_cc.append(float(r1['fpp_result'].get('fpp')) if r1['fpp_result'].get('fpp') is not None else np.nan)

x = np.arange(len(cfgs))

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10.5, 7.0), dpi=150, sharex=True)

ax1.plot(x, fpp_no, marker='o', label='No contrast curve')
ax1.plot(x, fpp_cc, marker='o', label='With contrast curve')
ax1.set_ylabel('FPP')
ax1.set_yscale('log')
ax1.grid(True, which='both', alpha=0.2)
ax1.legend(loc='upper right')
ax1.set_title('V24: FPP sensitivity (log scale)')

ax2.plot(x, depth_no, marker='o', label='Depth (no CC)')
ax2.plot(x, depth_cc, marker='o', label='Depth (with CC)')
ax2.set_ylabel('Depth used (ppm)')
ax2.grid(True, alpha=0.2)
ax2.legend(loc='upper right')

ax2.set_xticks(x)
ax2.set_xticklabels([_label(d) for d in cfgs], rotation=20, ha='right')

fig.tight_layout()
fig.savefig(run_path)
if docs_path is not None:
    fig.savefig(docs_path)
plt.close(fig)

print(
    json.dumps(
        {
            'run_plot_path': str(run_path),
            'docs_plot_path': str(docs_path) if docs_path is not None else None,
            'run_json_path': str(run_out_dir / 'triceratops_fpp_sensitivity_grid.json'),
            'docs_json_path': str(docs_out_dir / 'triceratops_fpp_sensitivity_grid.json') if docs_out_dir is not None else None,
        },
        indent=2,
        sort_keys=True,
    )
)


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/38_v24_triceratops_fpp_sensitivity_grid/V24_triceratops_fpp_sensitivity.png" width="980" />


<details>
<summary><b>Analysis</b></summary>

- **What this is probing:** whether the statistical FPP conclusion is *robust* to small, reasonable changes in preprocessing that alter the inferred depth.
- **How to read the plot:**
  - **Top panel (log FPP):** each point is one run. Lower is better; values below `0.01` are commonly considered statistically supportive.
  - **Bottom panel (depth used):** the depth that was actually passed into TRICERATOPS for that preprocessing choice.
  - Two curves compare **no AO constraint** vs **with AO contrast curve**.
- **Interpretation (this run):**
  - Across the tested settings, depths range from ~200 to ~241 ppm, showing that depth is still somewhat preprocessing-dependent.
  - Even so, FPP remains low across the grid; the **highest** tested no-contrast-curve case is still < `1e-3`.
  - Adding the AO contrast curve consistently pushes FPP lower (sometimes to ~0 within the Monte Carlo precision).
- **What is still a warning sign:** the spread in depth means the *numerical* FPP should be interpreted as conditional on preprocessing. If later analysis changes the depth meaningfully (or changes which sectors are trusted), re-run FPP using the updated baseline.
- **Practical next step:** choose and freeze a defensible preprocessing baseline (sector set + detrend) that produces a stable depth, then treat the corresponding FPP as the reportable value.

</details>
